In [126]:
import os
import time
import warnings
import datetime
import numpy as np
import pandas as pd
import random
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from scipy.optimize import minimize

import pylab

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap


from IPython.display import Image, SVG

%matplotlib inline

# 3. Реализация решающего дерева 

In [5]:
boston = datasets.load_boston()
bX_data, by_data = boston.data, boston.target
# Split data
X_train, X_test, y_train, y_test = train_test_split(bX_data, by_data, test_size=0.75, random_state=42)

In [26]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], 
      dtype='|S7')

In [44]:
None == 0

False

In [187]:
class Node:
    def __init__(self, num=None, value=None, color=None):
        self.color = color
        self.num = num
        self.value = value
        self.right_index = None
        self.left_index = None

class DecisionTree:    
    def __init__(self, max_depth=5, step_count = 10):
        self.step_count = step_count
        self.max_depth = max_depth
        self.nodes = []
    
    def _split_num_value(self, X, y, num, value):
        l_X = X[X[num] <= value]
        l_y = y[X[num][y.index] <= value]
        r_X = X[X[num] > value]
        r_y = y[X[num][y.index] > value]
        return l_X, l_y, r_X, r_y
    
    def predict(self, X_test):
        y = []
        # print(X_test)
        for _, x in X_test.iterrows():
            node_num = 0
            while not self.nodes[node_num].color:
                feature_num = self.nodes[node_num].feature_num
                feature_value = self.nodes[node_num].feature_value
                if x[feature_num] <= feature_value:
                    node_num = self.nodes[node_num].left_index
                else:
                    node_num = self.nodes[node_num].right_index
            y.append(self.nodes[node_num].color)
        return pd.Series(y)
        
    def how_different(self, y):
        # missclassification criteria
        if y.empty:
            return 1
        return min([(1. - 1.*len([label for label in y if label == label_example])/len(y)) for label_example in set(y)])
    
    def split(self, X, y, depth=0):
        print("len(X)={}\nlen(y)={}\ndepth={}\n\n".format(len(X), len(y), depth))
        node=Node()
        self.nodes.append(node)
        if depth > self.max_depth:
            node.color = np.argmax(np.bincount(y))
            return
        # bruteforce feature numbers and values
        min_cost = -1
        for i, (min_l, max_l) in enumerate(self.limits):
            step = (max_l - min_l)/(1.*self.step_count)
            value = min_l + step
            while value < max_l:
                # count value to minimize
                l_X, l_y, r_X, r_y = self._split_num_value(X, y, i, value)
#                 print(l_X)
#                 print(l_y)
#                 print(r_X)
#                 print(r_y)
                
                cost = len(l_X)*self.how_different(l_y) + len(r_X)*self.how_different(r_y)
                if min_cost < 0 or cost < min_cost:
                    optimal_l_X = l_X
                    optimal_l_y = l_y
                    optimal_r_X = r_X
                    optimal_r_y = r_y
                    node.feature_num = i
                    node.feature_value = value
                    min_cost = cost
                value += step
        #split
        if l_y.empty or r_y.empty:
            node.color = np.argmax(np.bincount(y))
            return
        node.left_index = len(self.nodes)
        self.split(l_X, l_y, depth+1)
        node.right_index = len(self.nodes)
        self.split(r_X, r_y, depth+1)
            
    
    def fit(self, X_train, y_train):
        self.limits = zip(np.amin(X_train, axis=0), np.amax(X_train, axis=0))
        self.nodes = []
        self.split(X_train, y_train)

In [106]:
german = pd.read_csv('german.data-numeric', header=None, delim_whitespace=True)

X_data = german[list(set(german.columns) - set([24]))]
y_data = german[24]

In [134]:
def build_tree(X_data, y_data, depth=None):
    global DecisionTree
    model = DecisionTree(max_depth=depth)
    model.fit(X_data, y_data)
    dot_data = tree.export_graphviz(model, out_file="small_tree.out", 
                             filled=True, rounded=True,  
                             special_characters=True)
    graph = pydotplus.graphviz.graph_from_dot_file("small_tree.out")  
    image = Image(graph.create_png())
    return image

In [ ]:
build_tree(X_data, y_data, 3)

In [146]:
from sklearn.metrics import accuracy_score

In [188]:
model = DecisionTree(max_depth=1)
model.fit(X_data, y_data)

len(X)=1000
len(y)=1000
depth=0


len(X)=370
len(y)=370
depth=1


len(X)=630
len(y)=630
depth=1




In [189]:
accuracy_score(y_data, model.predict(X_data))

0.69999999999999996

In [159]:
cross_val = []
train = []
x = np.arange(1,5)
for depth in x:
    print depth
    model = DecisionTree(max_depth=depth)
    model.fit(X_data, y_data)
    train.append(accuracy_score(y_data, model.predict(X_data)))
    print(train[-1])
    cross_val.append(cross_val_score(model, X_data, y_data, scoring='accuracy').mean())

1
len(X)=1000
len(y)=1000
depth=0


('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 297.0)
('cpst = ', 290.0)
('cpst = ', 300.0)
('cpst = ', 299.99999999999994)
('cpst = ', 299.0)
('cpst = ', 299.0)
('cpst = ', 290.00000000000006)
('cpst = ', 290.00000000000006)
('cpst = ', 283.0)
('cpst = ', 283.0)
('cpst = ', 299.99999999999994)
('cpst = ', 299.99999999999994)
('cpst = ', 299.99999999999994)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 299.99999999999994)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 299.0)
('cpst = ', 291.00000000000006)
('cpst = ', 288.0)
('cpst = ', 293.99999999999994)
('cpst = ', 297.0)
('cpst = ', 299.0)
('cpst = ', 299.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ', 300.0)
('cpst = ',

IndexError: tuple index out of range

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, cross_val, color='green')
plt.plot(x, train, color='blue')
plt.title("Cross-validation")
plt.show()